In [14]:
import os
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from torchvision import transforms
import torch.nn.functional as F
import random
from torchvision.transforms import v2
from matplotlib import colormaps
import io


In [2]:
!git clone https://github.com/facebookresearch/dinov3.git
%cd dinov3
!pip install -r requirements.txt

Cloning into 'dinov3'...
remote: Enumerating objects: 538, done.
remote: Counting objects: 100% (363/363), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 538 (delta 201), reused 99 (delta 99), pack-reused 175 (from 1)
Receiving objects: 100% (538/538), 9.88 MiB | 25.68 MiB/s, done.
Resolving deltas: 100% (223/223), done.
/content/dinov3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 35.5 MB/s eta 0:00:00


In [3]:
!wget "" -O "dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth"

--2025-12-17 10:28:52--  https://dinov3.llamameta.net/dinov3_vitb16/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiNXF0a3Q0djl5ZDZkMDlncXdmeno5MHB2IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZGlub3YzLmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjU5OTA0Nzh9fX1dfQ__&Signature=GOt0pzYq7KdFRXWv1yeCglKsLWadjmTN-ifqlB0gyJu4ZV5VQ8z2f%7EDW9KouJc7dmXCfblmaDt0PUcKR-6Qy%7EORLqF5LMaO8yy8jI3jhJDePq4w0qxAClyu7YYDaYOTvf4k%7E75PuWacsDBMPla5%7ExVspMQwuaYRNQc-qmshmukfCyGZdnljxj6EQIeI3qq5huyOcVyO-Jq9K78nf7HjWxBg2TUM%7Evw53hFDAn3yNQo%7EcsQaEoRgECa1nvYDhyb6j-qbhVhZHszojipU09CRAeuDfyeksPFs8B2TE2FXM8rXYaYSEnHfJ9s%7EhwneyohvZqS4XOOlZUy%7EIw--7-e5nHw__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1912522716015463
Resolving dinov3.llamameta.net (dinov3.llamameta.net)... 3.165.160.25, 3.165.160.69, 3.165.160.34, ...
Connecting to dinov3.llamameta.net (dinov3.llamameta.net)|3.165.160.25|:443... connected.
HTTP request sent, awaiting res

In [4]:
REPO_DIR = "/content/dinov3"
WEIGHTS = "/content/dinov3/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth"
dinov3_vitb16 = torch.hub.load(REPO_DIR, 'dinov3_vitb16', source='local', weights=WEIGHTS)

Downloading: "file:///content/dinov3/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth" to /root/.cache/torch/hub/checkpoints/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth


100%|██████████| 327M/327M [00:00<00:00, 711MB/s]


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dinov3_vitb16
dinov3_vitb16.eval()
dinov3_vitb16.to(device)

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (rope_embed): RopePositionEmbedding()
  (blocks): ModuleList(
    (0-11): 12 x SelfAttentionBlock(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): SelfAttention(
        (qkv): LinearKMaskedBias(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
    )
  )
  (norm): LayerN

In [35]:
def make_transform(resize_size: int = 2048):
    to_tensor = transforms.v2.ToImage()
    resize = transforms.v2.Resize((resize_size, resize_size), antialias=True)
    to_float = transforms.v2.ToDtype(torch.float32, scale=True)
    normalize = transforms.v2.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
    )
    return transforms.v2.Compose([to_tensor, resize, to_float, normalize])

def patch_embeddings(img, resize_dim):
    transform = make_transform(resize_dim).to(device)
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
      features_dict = dinov3_vitb16.forward_features(x)
      patch_features = features_dict['x_norm_patchtokens']
    return patch_features

def patch_map(patch_embeddings):
  patch_norms = patch_embeddings.norm(dim=-1).squeeze()
  grid_size = int(patch_norms.shape[0] ** 0.5)
  patch_map = patch_norms.reshape(grid_size, grid_size).cpu().numpy()
  return patch_map

def heatmap_overlay(image: Image.Image, patch_map: np.ndarray):
    # resize patch map to fit img
    heatmap = Image.fromarray(patch_map)
    heatmap = heatmap.resize(image.size, resample=Image.BILINEAR)
    heatmap = np.array(heatmap)

    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-6)

    fig, ax = plt.subplots()
    #ax.imshow(image)
    #ax.imshow(heatmap, cmap="jet", alpha=0.5)
    ax.imshow(heatmap)
    ax.axis("off")

    buf = io.BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight", pad_inches=0)
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)


def process_image(img):
    if not isinstance(img, Image.Image):
        img = Image.fromarray(img)

    patches = patch_embeddings(img, resize_dim=2048)
    pmap = patch_map(patches)

    heatmap_img = heatmap_overlay(img, pmap)
    patch_map_img = Image.fromarray(
        ((pmap - pmap.min()) / (pmap.max() - pmap.min()) * 255).astype(np.uint8)
    )

    embedding_stats = {
        "num_patches": patches.shape[1],
        "embedding_dim": patches.shape[2],
    }

    return img, patch_map_img, heatmap_img, embedding_stats

def extract_and_cache_tokens(img, resize_dim=2048):
    patches = patch_embeddings(img, resize_dim)[0]  # (N, D)
    patches = torch.nn.functional.normalize(patches, dim=1)

    grid_size = int(patches.shape[0] ** 0.5)
    return patches, grid_size

def pixel_to_patch_index(x, y, image_size, grid_size):
    H, W = image_size
    patch_h = H // grid_size
    patch_w = W // grid_size

    px = min(x // patch_w, grid_size - 1)
    py = min(y // patch_h, grid_size - 1)

    index = py * grid_size + px
    return index, px, py

def cosine_similarity_map(tokens, query_index, grid_size):
  query = tokens[query_index]        # (D,)
  similarities = tokens @ query      # (N,)
  sim_map = similarities.reshape(grid_size, grid_size)
  return sim_map.cpu().numpy()

def on_click(img, cached_tokens, cached_grid, evt: gr.SelectData):
    x, y = evt.index

    H, W = img.size[1], img.size[0]
    patch_idx, px, py = pixel_to_patch_index(
        x, y, (H, W), cached_grid
    )

    sim_map = cosine_similarity_map(
        cached_tokens, patch_idx, cached_grid
    )

    heatmap = heatmap_overlay(img, sim_map)

    info = {
        "clicked_pixel": (x, y),
        "patch_index": int(patch_idx),
        "patch_coords": (int(px), int(py)),
    }

    return heatmap, info



In [37]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## Patch Embeddings & Click-to-Cosine Similarity")
    cached_tokens = gr.State()
    cached_grid = gr.State()

    with gr.Row():
        input_img = gr.Image(
            type="pil",
            label="Input Image (click to query patch)", height=640
        )

    with gr.Row():
        patch_map_img = gr.Image(label="Norm map, 1 pixel = 1 ViT patch", height=1024)
        heatmap_out = gr.Image(label="Cosine Similarity Heatmap", height=640)

    info_out = gr.JSON(label="Click")

    def on_image_upload(img):
        tokens, grid = extract_and_cache_tokens(img, resize_dim=2048)
        pmap = patch_map(tokens.unsqueeze(0))
        pmap_img = Image.fromarray(
            ((pmap - pmap.min()) / (pmap.max() - pmap.min()) * 255).astype("uint8")
        )

        return tokens, grid, pmap_img

    input_img.change(
        fn=on_image_upload,
        inputs=input_img,
        outputs=[cached_tokens, cached_grid, patch_map_img],
    )

    input_img.select(
      fn=on_click,
      inputs=[input_img, cached_tokens, cached_grid],
      outputs=[heatmap_out, info_out],
    )


demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://adb69ac600e00d955e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://adb69ac600e00d955e.gradio.live
